# the perform a mcmc sampling for Table2 1993 QS  fe_13

## for the  markov-chain monte-carlo bayesian inference model of pymc3 developed by Chris Fonnesbeck, Anand Patil, David Huard, John Salvatier

## the original analysis was done with pymc version 2

## pymc3 is what is available now it is most easily obtained from conda-forge or mini-forge

In [ ]:
import os
import fnmatch
import json
import pickle
from datetime import date
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import pymc3 as pm
import theano
from theano import tensor as tt
import arviz as az
import ChiantiPy.core as ch
import ChiantiPy.tools.util as chutil
import ChiantiPy.model as mdl
import ChiantiPy.model.Maker as mm

In [ ]:
autoreload 3

In [ ]:
wd = os.getcwd()

In [ ]:
today = date.today()

In [ ]:
thisday =today.strftime('%Y_%B_%d')

In [ ]:
thisday

In [ ]:
matplotlib qt

In [ ]:
myIon = 'fe_13'

In [ ]:
nameDict = chutil.convertName(myIon)

In [ ]:
ls

## fe_13 data is in the .json file

In [ ]:
dirList = os.listdir(wd)

In [ ]:
jsonList = []
for fname in dirList:
    if fnmatch.fnmatch(fname,'*.json'):
        jsonList.append(fname)

In [ ]:
for idx, fname in enumerate(jsonList):
    print('idx:  %i  fname:  %s'%(idx, fname))

In [ ]:
jsonName = jsonList[0]

In [ ]:
jsonName

In [ ]:
with open(jsonName, 'r') as inpt:
    specData = json.load(inpt)

In [ ]:
specData.keys()

In [ ]:
specData['ref']

## the weighted chi-squared = sum ((pred-obs)/(wghtFactor x obs))**2

In [ ]:
wghtFactor = 0.2

## create the box of matches object

In [ ]:
box = mm.maker(specData, wghtFactor = wghtFactor, verbose = True)

In [ ]:
box.SpecData['filename']

In [ ]:
ls

## get the pickled match file name

In [ ]:
pklList = []
for fname in dirList:
    if fnmatch.fnmatch(fname,'*.pkl'):
        pklList.append(fname)

In [ ]:
for idx, fname in enumerate(pklList):
    print('idx:  %i  fname:  %s'%(idx, fname))

In [ ]:
matchName = pklList[0]

In [ ]:
matchName

## open the pickled match file

In [ ]:
with open(matchName,'rb') as inpt:
    match = pickle.load(inpt)

In [ ]:
match.keys()

## temperature and density are in the pickled match file

In [ ]:
match['EDensity'].shape

In [ ]:
' %10.2e'%(match['EDensity'][0])

In [ ]:
dens = match['EDensity']

In [ ]:
' density range = %10.2e to %10.2e'%(dens.min(), dens.max())

## load the matches saved in the pickle file

In [ ]:
box.loadMatch(matchName)

## demo #1 gives a prettier plot

In [ ]:
plt.figure()
box.emPlot(vs='d')
plt.text(1.e+7, 3.e+26, myIon, horizontalalignment='center', fontsize=16)
plt.xlabel('Electron Density (cm$^{-3}$)', fontsize=14)
plt.ylabel('Emissin Measure (cm$^{-5}$)', fontsize=14)
plt.tight_layout()

### a plot of intensity or Gofnt(T) - the usual EM loci plot but using box.emPlot

In [ ]:
plt.figure()
box.emPlot(vs='d')
plt.text(1.e+7, 2.e+26, nameDict['spectroscopic'], fontsize=16)
plt.title('T = %10.2e'%(box.Temperature[0]))
plt.tight_layout()

## from the plots, a density of 7.e+8 and an em of 1.e+27 is estimated

In [ ]:
nDens = box.EDensity.size
print(' # of density values %5i'%(nDens))

In [ ]:
densGuess = 7.e+8
Dindex = int(np.argmin(np.abs(box.EDensity - densGuess)))
' Dindex = %5i'%(Dindex)

### by setting the temperature by its index and the log of the EM, we can predict the spectrum

In [ ]:
box.emSetIndices(Dindex)
print('density set to %10.2e '%(box.EDensity[Dindex]))

In [ ]:
emGuess = 1.e+27
emLog = np.log10(emGuess)

In [ ]:
box.emSetIndices(Dindex)

In [ ]:
box.emSet(emLog)

## Dindex and emLog will be the starting points in the MCMC sampling

## want an em Min and Max for the MCMC sampling, guessed from the plots

In [ ]:
emMinMax = [3.e+26,3.e+27]
emLogMinMax = np.log10(emMinMax)

### get predicted intensities from initial guess

In [ ]:
box.predict()

## can print out the predicted intensities, as in NB #3

In [ ]:
box.predictPrint(filename='priorPredictPrint.txt')

## begin set up for mcmc sampling

In [ ]:
nDens = match['EDensity'].size
print(' # of densities %5i'%(nDens))

## tune and samples may need to be increased after the first run and then rerun

## the no. of cores depends on your machine

In [ ]:
tune = 2000
samples = 100000
cores = 4

In [ ]:
padding = 10

## the predicted intensity matrix

In [ ]:
pred = np.zeros((nDens, len(box.Match)), np.float64)
for iwvl, amatch in enumerate(box.Match):
    pred[0:,iwvl] += amatch['intensitySum']

In [ ]:
' tune = %i  samples = %i cores = %i'%(tune, samples, cores)

## the observed intensities

In [ ]:
nObs = len(box.Match)
intensity = np.zeros(nObs, np.float64)
for iwvl in range(nObs):
    intensity[iwvl] = box.Match[iwvl]['obsIntensity']

## create the MCMC model and perform the sampling

In [ ]:
with pm.Model() as model:
    d0 = pm.DiscreteUniform('d0', lower = 0, upper = nDens - 1, dtype='int64', testval = Dindex)
    
    em = pm.Uniform('em', lower=emLogMinMax[0], upper= emLogMinMax[1], testval = emLog)  #  sigma was 0.1

    xpred = theano.shared(pred, name='p0')

    idx0 = tt.as_tensor_variable(d0)
    
    predicted = xpred[idx0]*10.**em
    
    sigma = np.sqrt(wghtFactor*intensity)
    
    Y_obs = pm.Normal('Y_obs', mu=predicted, sigma=sigma, observed=intensity)

    pm.NUTS([em],target_accept=0.87)  # was 0.8 was 0.87
    pm.Metropolis([d0], target_accept=0.87)  # was 0.8 was 0.87
    start = {'d0':Dindex}
    start['em'] = emLog
    trace = pm.sample(samples, tune=tune, cores=cores, start=start)
    pm_data = az.from_pymc3(
        trace=trace)
pm_data


In [ ]:
pm_data.posterior.keys()

In [ ]:
pm_data.posterior['em'].shape

In [ ]:
pm_data.posterior['d0'].shape

In [ ]:
pm_data.log_likelihood.keys()

In [ ]:
pm_data.sample_stats.keys()

In [ ]:
pm_data.observed_data.keys()

## save the trace for later analysis

## here, the trace will be used for analysis and not the pm_data

In [ ]:
pth = pm.save_trace(trace)

In [ ]:
pth

## can come back later an load the trace but it will need the model available 

## can create a model that skips the sampling

trace = pm.load_trace(pth, model)

## arviz can plot the traces and histograms, among other things

In [ ]:
az.plot_trace(pm_data, var_names=["d0"])

In [ ]:
az.plot_trace(pm_data, var_names=["em"])

In [ ]:
trace.stat_names

In [ ]:
trace.varnames

In [ ]:
trace['d0'].shape

## get the mean values and std of the traces

In [ ]:
d0Mean = trace['d0'].mean()
d0Std = trace['d0'].std()
' d0Mean = %i d0Std = %10.5f'%(d0Mean, d0Std)

## Dindex was our initial guess

In [ ]:
Dindex

In [ ]:
plt.figure()
xyhist = plt.hist(trace['d0'])
plt.xlabel('Temperature Index', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.title('d0    mean = %8.3f  std = %8.5f \n tune = %i  samples = %i wF:  %6.3f'%(d0Mean, d0Std, tune, samples, wghtFactor), fontsize=14)
plt.tight_layout()

In [ ]:
em0Mean = trace['em'].mean()
em0Std = trace['em'].std()
' em0Mean = %10.3f em0Std = %10.5f'%(em0Mean, em0Std)

## emLog was our initial guess

In [ ]:
emLog

In [ ]:
plt.figure()
xyhist = plt.hist(trace['em'])
plt.xlabel('Log$_{10}$ Emission Measure', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.title('EM    mean = %8.3f  std = %8.5f \n tune = %i  samples = %i wF: %6.3f'%(em0Mean, em0Std, tune, samples, wghtFactor), fontsize=14)
plt.tight_layout()

## save the results to a file

In [ ]:
results = {}
results['dMean'] = d0Mean
results['dStd'] = d0Std
results['emMean'] = em0Mean
results['emStd'] = em0Std
results['tune'] = tune
results['samples'] = samples
results['date'] = thisday
results['wghtFactor'] = wghtFactor
results['pth'] = pth
results['emMinMax'] = emMinMax
results['emLogMinMax'] = emLogMinMax

In [ ]:
resultsName = 'results_%s_%i_%i.pkl'%(thisday,tune,samples)

In [ ]:
resultsName

In [ ]:
with open(resultsName,'wb') as outpt:
    pickle.dump(results,outpt)

## predict the intensities with the parameters derived from the MCMC sampling

In [ ]:
newDindex = int(np.round(d0Mean))

In [ ]:
newDindex

In [ ]:
' newDindex %i  priorDindex  %i'%(newDindex, Dindex)

In [ ]:
newEmLog = em0Mean

In [ ]:
' new Emlog:  %10.3f   prior EMlog:  %10.3f'%(newEmLog, emLog)

In [ ]:
box.emSetIndices([newDindex])
print('density  set to %10.2e '%(dens[newDindex]))

In [ ]:
box.EmIndices

In [ ]:
box.emSet([newEmLog])

In [ ]:
box.predict()

In [ ]:
sort = 'wvl'

In [ ]:
matchName

In [ ]:
os.path.splitext(matchName)

In [ ]:
printName = os.path.splitext(matchName)[0] + '_postPredictPrint_%i_%i_%s_%s.txt'%(tune, samples, today, sort)

In [ ]:
printName

In [ ]:
box.predictPrint(filename=printName, sort=sort)

In [ ]:
diffName = os.path.splitext(matchName)[0] + '_postDiffPrint_%i_%i_%s_%s.txt'%(tune, samples, today, sort)

In [ ]:
diffName

In [ ]:
box.diffPrint(filename=diffName, sort=sort)

## plot the differences

In [ ]:
wvl = box.Diff['wvl']
diff = box.Diff['diffOverInt']

In [ ]:
diffMean = diff.mean()
diffStd = diff.std()

In [ ]:
mytitle = 'diff Mean %10.3f  diff Std  %10.3f'%(diffMean, diffStd)

In [ ]:
mytitle

In [ ]:
plt.figure()
plt.plot(wvl, diff,'o')

In [ ]:
plt.axhline(diffMean, color='k', lw=2, label='Mean')
plt.axhline(diffMean + diffStd, color='r', lw=2, linestyle='--', label='1 std')
plt.axhline(diffMean - diffStd, color='r', lw=2, linestyle='--')  #, label='1 std')
plt.axhline(diffMean + 2.*diffStd, color='b', lw=2, linestyle='dotted', label='2 std')
plt.axhline(diffMean - 2.*diffStd, color='b', lw=2, linestyle='dotted')  #, label='2 std')
plt.axhline(diffMean + 3.*diffStd, color='g', lw=2, linestyle='dotted', label='3 std')
plt.axhline(diffMean - 3.*diffStd, color='g', lw=2, linestyle='dotted')  #, label='3 std')

In [ ]:
plt.xlabel('Wavelength ($\AA$)', fontsize=14)
plt.ylabel(r'(Obs - Pred)/(w $\times$ Obs)', fontsize=14)

In [ ]:
plt.title(mytitle, fontsize=14)

In [ ]:
plt.legend(loc='upper right', bbox_to_anchor=(0.99, 1.0), fontsize=12)

In [ ]:
plt.tight_layout()

## this can also be done with diffPlot

In [ ]:
box.diffPlot(title=True)

## the plot object are saved in the DiffPlot attribute

In [ ]:
box.DiffPlot.keys()

In [ ]:
fig = box.DiffPlot['fig']
ax =  box.DiffPlot['ax']

## so, can do things like the following

In [ ]:
ax.set_xlim([310., 330.])

In [ ]:
fig.tight_layout()